# Part 17

# Using Functions

Universal code for the entire notebook

In [185]:
# Uncomment the line below to make sure you have all the packages needed
# %pip install -r requirements.txt

In [186]:
# Standard library imports

# Third-party library imports
from openai import OpenAI  # Used for interacting with OpenAI's API
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants
from typing_extensions import override  # Used for overriding methods in subclasses
from IPython.display import display, Markdown, clear_output  # Used for displaying content in Jupyter Notebooks


In [187]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

In [188]:
# Define the functions the assistant will call
def get_current_temperature():
    # Simulate getting the current temperature
    return 57

def get_rain_probability():
    # Simulate getting the rain probability
    return 0.06

In [189]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize the results list
        self.last_text_created = None  # To store the last created text
        self.last_text_delta = None  # To store the last text delta

    @override
    def on_text_created(self, text) -> None:
        """Handle the event when text is first created."""
        self._handle_text_created(text)

    @override
    def on_event(self, event):
        """Handle generic events."""
        self._handle_event(event)

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        self._handle_text_delta(delta)

    def on_tool_call_created(self, tool_call):
        """Handle the event when a tool call is created."""
        self._handle_tool_call_created(tool_call)

    def on_tool_call_delta(self, delta, snapshot):
        """Handle the event when there is a delta (update) in a tool call."""
        self._handle_tool_call_delta(delta)

    def _handle_text_created(self, text):
        """Internal method to handle text creation."""
        if text != self.last_text_created:  # Check if the text is different from the last created text
            print("\nassistant text > ", end="", flush=True)
            self.results.append(text)
            self.last_text_created = text  # Update the last created text

    def _handle_event(self, event):
        """Internal method to handle events."""
        if event.event == 'thread.run.requires_action':
            run_id = event.data.id
            self._handle_requires_action(event.data, run_id)

    def _handle_text_delta(self, delta):
        """Internal method to handle text deltas."""
        if delta.value != self.last_text_delta:  # Check if the delta is different from the last text delta
            print(delta.value, end="", flush=True)
            self.results.append(delta.value)
            self.last_text_delta = delta.value  # Update the last text delta

    def _handle_tool_call_created(self, tool_call):
        """Internal method to handle tool call creation."""
        print(f"\nassistant tool > {tool_call.type}\n", flush=True)

    def _handle_tool_call_delta(self, delta):
        """Internal method to handle tool call deltas."""
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
                self.results.append(delta.code_interpreter.input)
            if delta.code_interpreter.outputs:
                print("\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs or []:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)
                        self.results.append(output.logs)

    def _handle_requires_action(self, data, run_id):
        """Internal method to handle requires action events."""
        tool_outputs = []
        for tool in data.required_action.submit_tool_outputs.tool_calls:
            if tool.function.name == "get_current_temperature":
                temperature = get_current_temperature()
                tool_outputs.append({"tool_call_id": tool.id, "output": str(temperature)})
            elif tool.function.name == "get_rain_probability":
                rain_probability = get_rain_probability()
                tool_outputs.append({"tool_call_id": tool.id, "output": str(rain_probability)})
        self._submit_tool_outputs(tool_outputs, run_id)

    def _submit_tool_outputs(self, tool_outputs, run_id):
        """Internal method to submit tool outputs."""
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            for text in stream.text_deltas:
                print(text, end="", flush=True)


In [190]:

assistant = client.beta.assistants.create(
    instructions="You are a weather bot. Use the provided functions to answer questions.",
    model="gpt-4o",
    tools=[
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get the current temperature for a specific location",
            "parameters": {
            "type": "object",
            "properties": {
                "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
                },
                "unit": {
                "type": "string",
                "enum": ["Celsius", "Fahrenheit"],
                "description": "The temperature unit to use. Infer this from the user's location."
                }
            },
            "required": ["location", "unit"]
            }
        }
        },
        {
        "type": "function",
        "function": {
            "name": "get_rain_probability",
            "description": "Get the probability of rain for a specific location",
            "parameters": {
            "type": "object",
            "properties": {
                "location": {
                "type": "string",
                "description": "The city and state, e.g., San Francisco, CA"
                }
            },
            "required": ["location"]
            }
        }
        }
    ]
)


In [191]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather in San Francisco today and the likelihood it'll rain?",
)

In [192]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler()
    ) as stream:
    stream.until_done()


assistant tool > function


assistant tool > function


assistant text > TheThe current current temperature temperature in in San San Francisco Francisco,, CA CA is is  5757°F°F,, and and the the likelihood likelihood of of rain rain today today is is  66%.%.